In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

In [2]:
ws = Workspace.from_config()

In [7]:
# compute target - you can use different compute targets in different steps in the pipeline
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

pipeline_cluster = "Demo-Compute-Cluster"
compute_target = ws.compute_targets[pipeline_cluster]

In [8]:
import pandas as pd

data = pd.read_csv(
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
)
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [10]:
#we will store intermediate data between data preparation step and autoML step in the default datastore of the workspace.
datastore = ws.get_default_datastore()

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

dataset = TabularDatasetFactory.register_pandas_dataframe(
    data, target=(datastore, "dataset/"), name="AutoMLE2ETraininggPipeline_Classification_dataset"
)
#I creat dataset from pandas dataframe.
#Other way: You can create  tabular dataset by using ds=Dataset.Tabular.from_delimited_files(path=xxx) and register it with ds.register()

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to dataset//b4890d23-2d45-4c4e-936f-93d606e4f5f2/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [13]:
#Target: Configure the training run- Environment setup

from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

#env = Environment.get(workspace=ws, name='experiment_env', version='2')
curated_env=Environment.get(workspace=ws, name='AzureML-sklearn-1.0-ubuntu20.04-py38-cpu')
pipeline_run_config=RunConfiguration()
pipeline_run_config.environment=curated_env

# Add conda dependencies to the automl env:
#pipeline_run_config.environment.python.conda_dependencies = CondaDependencies.create(
 #conda_packages=['pandas'])  

In [15]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

#1st way: OutputFileDatasetConfig without giving destination
#prepped_output_path = OutputFileDatasetConfig(name="output_path") 
# OutputFileDatasetConfig  points a directory and CSV file will be written there. Also given as parameter below.
# If you do not give destination parameter, it will copy the output to the workspaceblobstore datastore, under the path /dataset/{run-id}/{output-name}. 
# Run-id is the name value on the overview of the job and outputname is the name given as a parameter. In my example, it is output_path.

#2nd way: OutputFileDatasetConfig with giving destination
output_path = (datastore, f"azureml/classification_prep_output/")
prepped_output_path = OutputFileDatasetConfig(destination = output_path)

input_ds = Dataset.get_by_name(ws, 'AutoMLE2ETraininggPipeline_Classification_dataset')

prep_step=PythonScriptStep(
    name="Prepare AutoML Classification",
    script_name="prepare.py",
    source_directory="./Scripts",
    #arguments=['--input-data',input_ds.as_named_input('AutoMLE2ETraininggPipeline_Classification_dataset'),'--prepped-data',output],
    arguments=["--output_path",prepped_output_path],
    inputs=[input_ds.as_named_input('AutoMLE2ETraininggPipeline_Classification_dataset')],
    compute_target=compute_target,
    runconfig=pipeline_run_config
)

In [16]:
#In an ML pipeline, the input data must be a Dataset object.

prepped_test_data=prepped_output_path.read_delimited_files("prepped_train_data_classification.csv") # This is the test data to send automlstep
prepped_train_data = prepped_output_path.read_delimited_files("prepped_test_data_classification.csv")

from azureml.pipeline.core import TrainingOutput,PipelineData

metrics_data = PipelineData(name='metrics_data',
                            datastore=datastore,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='best_model_data',
                          datastore=datastore,
                          pipeline_output_name='model_output',
                          training_output=TrainingOutput(type='Model'))
# two pipelinedata objects for automl outputs: metrics and the model.


In [17]:
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 60,
    "iterations" : 50,
    "experiment_timeout_hours" : 0.25,
    "primary_metric" : 'AUC_weighted'
}
#the run will stop after 50 iterations or 60 minutes, whichever comes first.

automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target= compute_target,
                           #  run_configuration = pipeline_run_config,
                             featurization = 'auto',
                             training_data = prepped_train_data,
                             test_data=prepped_test_data,
                             label_column_name = 'y',
                             **automl_settings)
# debug_log local file if you want to see logs

train_step = AutoMLStep(name='AutoML_Classification',
    automl_config=automl_config,
    outputs=[metrics_data,model_data],
    enable_default_model_output=False,
    enable_default_metrics_output=False,
    allow_reuse=True)

In [18]:
from azureml.pipeline.core.graph import PipelineParameter

# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="AutoML_Classification_Test")

register_step = PythonScriptStep(
     name="register_model",
     script_name="register_model.py",
     source_directory="./Scripts",
     allow_reuse=False,
     arguments=["--model_name", model_name, "--model_path", model_data],
     inputs=[model_data],
     compute_target=compute_target,
     runconfig=pipeline_run_config)

In [19]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

experiment = Experiment(ws, name= "automl-classification-E2E_trainingPipeline")

pipeline = Pipeline(ws, [prep_step, train_step,register_step])
run = experiment.submit(pipeline, show_output=True)
run.wait_for_completion()

Created step Prepare AutoML Classification [cb9248a2][51eac883-f61b-4a11-8ce4-d83a24d831f7], (This step will run and generate new outputs)
Created step AutoML_Classification [605ef40f][b2ec055c-7531-4226-8f3d-011adfdf6609], (This step will run and generate new outputs)Created step register_model [746ac52d][ad535f76-59c6-48f5-9eb2-fda5669c6b94], (This step will run and generate new outputs)

Submitted PipelineRun 60c71a86-23d5-4a35-804b-d70e2acecc93
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/60c71a86-23d5-4a35-804b-d70e2acecc93?wsid=/subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourcegroups/prodrg/workspaces/prod&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
PipelineRunId: 60c71a86-23d5-4a35-804b-d70e2acecc93
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/60c71a86-23d5-4a35-804b-d70e2acecc93?wsid=/subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourcegroups/prodrg/workspaces/prod&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
PipelineRun Status:

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process 'python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error: Traceback (most recent call last):\\n  File \\\"prepare.py\\\", line 34, in <module>\\n    train_ds=run.input_datasets['AutoMLE2EPipeline_Classification_train']\\nKeyError: 'AutoMLE2EPipeline_Classification_train'\\n\\n'. Please check the log file 'user_logs/std_log.txt' for more details.\"}\n{\n  \"code\": \"ExecutionFailed\",\n  \"target\": \"\",\n  \"category\": \"UserError\",\n  \"error_details\": [\n    {\n      \"key\": \"exit_codes\",\n      \"value\": \"1\"\n    }\n  ]\n}",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \